In [33]:
import numpy as np

def solve_triangular_inferior(L, B):
    """Resolve um sistema triangular inferior do tipo Ux + B."""

    n = L.shape[0]          # Ordem da matriz

    # Cópias usuais para evitar problemas
    X = B.copy().reshape(n)

    # Vai linha-a-linha, de ****cima para baixo***, escalonando a matriz utilizando o pivô na diagonal
    for i in range(n):
        for j in range(i):
            X[i] -= L[i,j] * X[j] # Pivoteia a i-ésima coluna, utilizando a entrada diagonal como pivô
        X[i] /= L[i,i]     # Normaliza a i-ésima linha

    return X


Para testar essa função, vamos criar uma matriz triangular inferior $L$ e um vetor $x$ aleatórios, e verificar se o retorno $x$ dessa função satisfaz $Ly=x$.

In [42]:
L = 20*np.random.rand(4,4) -10 # Matriz triangular aleatória com entradas em [-10,10]

for i in range(4):
    for j in range(i):
        L[j,i]=0.0        # Aniquila as entradas abaixo da diagonal principal para deixar triangular inferior
        
        
b = 20*np.random.rand(4) - 10 # Vetor aleatório com entradas em [-10,10]

y = solve_triangular_inferior(L,b) # Solução pretendida do sistema Ux = b

print(f"\nA matriz L é dada por\n{L}")
print(f"\nO vetor b é dada por\n{b}")
print(f"\nO vetor y é dada por\n{y}")
print(f"\nO vetor Lx é dada por\n{L@y}")


A matriz L é dada por
[[-6.87706406  0.          0.          0.        ]
 [ 8.91522971  4.16946918  0.          0.        ]
 [-7.17137731 -5.8557376  -8.22289805  0.        ]
 [-7.30279921  5.47971039  4.4530871   4.56334494]]

O vetor b é dada por
[-8.32543546  5.06935459 -1.0206757  -0.18596201]

O vetor y é dada por
[ 1.21060897 -1.3727173   0.04587365  3.50021451]

O vetor Lx é dada por
[-8.32543546  5.06935459 -1.0206757  -0.18596201]
